# 20日动量阈值 Top1 ETF 轮动策略（T+1 开盘执行版）

> 本策略在固定的 11 只 ETF 池中，使用 **20 日区间涨幅**作为动量信号，设置动量阈值 **[5%, 100%]** 作为入场门控；  
> 每次只持有 **1 只** ETF，并设置 **最小持有期 3 天**、**止盈 14%** 与 **止盈后冷却 6 天**。  
> **执行假设：收盘计算信号，下一交易日开盘成交（trade next open）**。

---

## 1. Universe（ETF Pool）

固定 11 只 ETF：
- 纳斯达克ETF (513300)
- 德国ETF (513030)
- 日经ETF (513520)
- 创业板100ETF (159915)
- 黄金ETF (518880)
- 石油LOF (162719)
- 科技ETF (515000)
- 酒ETF (512690)
- 红利低波ETF (512890)
- 中概互联网ETF (513050)
- 国投白银LOF (161226)

---

## 2. Data & Timing（数据与时序）

- 使用日线收盘价 `Close_i(t)` 计算信号。
- **决策时点**：交易日 `t` 收盘后（基于当日收盘数据）。
- **执行时点**：交易日 `t+1` 开盘按目标权重成交。

> 即：`signals@close(t) -> trades@open(t+1)`，避免不现实的当日收盘成交假设。

---

## 3. Signal（动量信号）

### 3.1 20日动量（区间涨幅）
对每只 ETF \(i\)，在 `t` 日收盘计算：

\[
mom_i(t) = \frac{Close_i(t)}{Close_i(t-20)} - 1
\]

---

## 4. Eligibility Filter（动量阈值 + 冷却过滤）

### 4.1 动量阈值门控
只有当：

\[
0.05 \le mom_i(t) \le 1.00
\]

该 ETF 才进入候选集合。

### 4.2 止盈冷却门控
若 ETF \(i\) 在最近一次止盈后仍处于冷却期（冷却 6 个交易日），则 **禁止买入**：

\[
t \le cooldown\_until_i \Rightarrow i \notin candidates(t)
\]

---

## 5. Selection（Top1 选择）

设候选集合：
\[
C(t)=\{ i \in U : 0.05 \le mom_i(t) \le 1.00,\ \text{and not in cooldown}\}
\]

- 若 \(C(t)\neq\emptyset\)：选择动量最大的 1 只 ETF：
\[
target(t)=\arg\max_{i\in C(t)} mom_i(t)
\]

- 若 \(C(t)=\emptyset\)：目标持仓为空（`target=None`），组合进入 **空仓（现金）** 状态。

---

## 6. Holding Constraint（最小持有期 3 天）

- 每次买入形成一笔持仓，记录 `entry_day`。
- 若当前持仓未满 3 个交易日，则 **禁止换仓**（但止盈可打断，见下一节）：

\[
hold\_days(t) < 3 \Rightarrow \text{ignore } target(t) \text{ rotation}
\]

---

## 7. Take-Profit Overlay（收益止盈 14% + 冷却 6 天）

### 7.1 止盈判定（在 t 日收盘检查）
若当前持仓 ETF 为 \(j\)，入场价格为 \(EntryPrice_j\)，则持仓收益：

\[
ret_j(t)=\frac{Close_j(t)}{EntryPrice_j}-1
\]

触发条件：
\[
ret_j(t)\ge 0.14 \Rightarrow \text{take profit trigger at } t
\]

### 7.2 止盈执行（t+1 开盘卖出）
- 若 t 日收盘触发止盈，则在 **t+1 开盘**将当前持仓清零（卖出）。
- 卖出后设置冷却期：
\[
cooldown\_until_j = (t+1) + 6 \text{ trading days}
\]

> 冷却从卖出后的下一交易日起计，冷却期内该 ETF 不可再买。

---

## 8. Priority of Rules（规则优先级）

在 `t` 日收盘生成 `t+1` 的交易指令时，优先级如下（从高到低）：

1) **止盈优先**：若当前持仓触发止盈，则 `t+1` 开盘先卖出该持仓，并进入冷却。  
2) **最小持有期**：若未触发止盈且持有天数 < 3，则 `t+1` 不允许因轮动而换仓（继续持有）。  
3) **动量轮动**：若满足最小持有期且候选集合非空，则调仓至 `target(t)`。  
4) **候选为空**：若无候选标的，则在 `t+1` 开盘空仓（或维持空仓）。

---

## 9. Portfolio Construction（组合权重）

- 单一持仓（Top1）：  
  - 若存在 `target(t)` 且未被持有期/止盈规则阻止，则：
    - `w_target(t+1) = 1.0`（满仓）
  - 否则：
    - `w_cash(t+1) = 1.0`（空仓）

---

## 10. Execution（Trade Next Open）

在 `t+1` 开盘执行以下目标权重变更（目标来自 `t` 收盘信号）：

- 若需要换仓：  
  - 卖出非目标持仓至 0  
  - 买入目标 ETF 至 100%（用可用现金）
- 若止盈触发：  
  - 卖出当前持仓至 0  
  - 该交易日不再买回同一 ETF（且未来 6 日禁止买入）

---

## 11. Notes（回测注意事项）

- 佣金与滑点若设为 0，会显著高估表现；该策略因日频监控 + 轮动，换手可能较高。
- `trade next open` 会比“当日成交/收盘成交”更保守，通常年化更低、回撤更真实。
- 对 LOF/跨市场 ETF 需注意跟踪误差与流动性（在真实执行中会影响开盘成交价与滑点）。


In [ ]:
from dataclasses import dataclass
from typing import Optional, Dict
import numpy as np
import pandas as pd

@dataclass(frozen=True)
class MomTop1Config:
    lookback: int = 20
    mom_low: float = 0.05
    mom_high: float = 1.00

    min_hold_days: int = 3      # 最小持有期（交易日）
    take_profit: float = 0.14   # 14%
    tp_cooldown_days: int = 6   # 止盈后冷却 6 个交易日（卖出后 next 6 opens 禁止买回）


def compute_mom_ret(close: pd.DataFrame, lookback: int = 20) -> pd.DataFrame:
    close = close.sort_index()
    mom = close / close.shift(lookback) - 1.0
    return mom


def build_mom_threshold_top1_weights_next_open(
    md,
    cfg: MomTop1Config,
    *,
    tickers: Optional[list[str]] = None,
) -> dict:
    """
    Outputs decision-time weights indexed by close dates (UNSHIFTED).
    With entry_mode="next_open" (properly aligned as described), these weights
    represent trades at next open.

    Returns dict with:
      - weights: DataFrame
      - mom: momentum DataFrame (signal)
      - top1_table: per decision date diagnostics
    """
    close = md.close.sort_index()
    opn = md.open.sort_index().reindex_like(close)

    if tickers is not None:
        close = close[tickers]
        opn = opn[tickers]

    dates = close.index
    cols = list(close.columns)

    mom = compute_mom_ret(close, lookback=cfg.lookback)

    # decision weights at close[t]
    w = pd.DataFrame(0.0, index=dates, columns=cols)

    # --- state ---
    pos_ticker: Optional[str] = None
    entry_exec_date: Optional[pd.Timestamp] = None   # the open date when we actually bought
    entry_price: float = np.nan

    # cooldown_end_pos[ticker] = last TRADE-DATE index position that is blocked (trade-date = open[t+1])
    cooldown_end_pos: Dict[str, int] = {t: -10**9 for t in cols}

    # helper: map date -> position
    date_pos = pd.Series(np.arange(len(dates)), index=dates)

    # diagnostics
    rows = []

    for dt in dates:
        p = int(date_pos.loc[dt])

        # trade date is next day open (dt_next)
        if p + 1 >= len(dates):
            # no next open -> cannot execute anything
            w.loc[dt] = 0.0 if pos_ticker is None else w.loc[dt]  # keep as-is; last day won't matter much
            rows.append({"date": dt, "target": None, "pos": pos_ticker, "mom_top": np.nan,
                         "take_profit_hit": False, "hold_days": 0, "action": "eof"})
            continue

        dt_next = dates[p + 1]     # execution happens at open[dt_next]
        trade_pos = p + 1          # index position of trade date

        # --- 1) compute candidates based on momentum threshold + cooldown ---
        mom_row = mom.loc[dt].replace([np.inf, -np.inf], np.nan)

        # threshold gate
        thr_ok = (mom_row >= cfg.mom_low) & (mom_row <= cfg.mom_high)

        # cooldown gate uses TRADE DATE (dt_next open) as the re-entry moment
        cd_ok = pd.Series(True, index=cols)
        for tkr in cols:
            cd_ok.loc[tkr] = trade_pos > cooldown_end_pos[tkr]

        eligible = thr_ok & cd_ok & mom_row.notna()

        candidates = mom_row[eligible]
        target = None
        mom_top = np.nan
        if len(candidates) > 0:
            target = candidates.idxmax()
            mom_top = float(candidates.loc[target])

        # --- 2) priority rules ---
        # 2.1 take-profit has highest priority (checked at close[dt])
        take_profit_hit = False
        hold_days = 0

        if pos_ticker is not None and entry_exec_date is not None:
            # how many trading days held as-of close[dt]
            hold_days = int(date_pos.loc[dt] - date_pos.loc[entry_exec_date] + 1)

            # take-profit check: close vs entry_open price
            px_now = float(close.loc[dt, pos_ticker])
            ret_since_entry = px_now / float(entry_price) - 1.0
            if ret_since_entry >= cfg.take_profit:
                take_profit_hit = True

        action = None

        if take_profit_hit:
            # at open[dt_next], we exit to cash: weights[dt] should be 0
            w.loc[dt] = 0.0

            # set cooldown for this ticker: block next `tp_cooldown_days` trade dates AFTER sell
            # sell happens at open[dt_next] (trade_pos). Block trade_pos+1 ... trade_pos+tp_cooldown_days
            cooldown_end_pos[pos_ticker] = trade_pos + cfg.tp_cooldown_days

            # reset position state (effective from dt_next open)
            pos_ticker = None
            entry_exec_date = None
            entry_price = np.nan

            action = "take_profit_exit_to_cash"

        else:
            # 2.2 min-hold: if in position and hold_days < min_hold_days, ignore rotation (keep holding)
            if pos_ticker is not None and hold_days < cfg.min_hold_days:
                # keep holding -> set weight for current pos at dt
                w.loc[dt] = 0.0
                w.loc[dt, pos_ticker] = 1.0
                action = "hold_min_hold"

            else:
                # 2.3 rotation: choose target if exists else cash
                if target is None:
                    w.loc[dt] = 0.0
                    action = "no_candidate_cash"
                    # if we had a position, this implies strategy-exit at dt_next open
                    pos_ticker = None
                    entry_exec_date = None
                    entry_price = np.nan
                else:
                    # switch/enter target
                    w.loc[dt] = 0.0
                    w.loc[dt, target] = 1.0
                    action = "enter_or_switch_target"

                    # update entry state only if changing position (or entering from cash)
                    if pos_ticker != target:
                        pos_ticker = target
                        entry_exec_date = dt_next           # bought at open[dt_next]
                        entry_price = float(opn.loc[dt_next, target])

        rows.append({
            "date": dt,
            "target": target,
            "pos": pos_ticker,
            "mom_top": mom_top,
            "take_profit_hit": take_profit_hit,
            "hold_days": hold_days,
            "action": action,
        })

    top1_table = pd.DataFrame(rows).set_index("date")
    return {"weights": w, "mom": mom, "top1_table": top1_table}


In [ ]:
from qresearch.backtest.portfolio import backtest_weights, plot_compare
from qresearch.backtest.config import ExperimentConfig
from qresearch.data.yfinance import download_market_data

cfg = ExperimentConfig(
    start="2021-01-01",
    end=None,
    entry_mode="next_close",          # IMPORTANT
    fee_bps=0.0,
    rf_annual=0.0,
    benchmark_mode="single_ticker",
    benchmark_ticker="513300.SS",
)

tickers = [
    '513300.SS', '513030.SS', '513520.SS', '159915.SZ', '518880.SS',
    '162719.SZ', '512890.SS', '513050.SS',
]

md = download_market_data(tickers, start=cfg.start, end=cfg.end)

固定 11 只 ETF：
- 纳斯达克ETF (513300)
- 德国ETF (513030)
- 日经ETF (513520)
- 创业板100ETF (159915)
- 黄金ETF (518880)
- 石油LOF (162719)
- 科技ETF (515000)
- 酒ETF (512690)
- 红利低波ETF (512890)
- 中概互联网ETF (513050)
- 国投白银LOF (161226)

In [ ]:
(1 + md.close.pct_change()).cumprod().plot(legend=True, figsize=(10, 6))

In [ ]:
from qresearch.backtest.portfolio import build_benchmark_weights

mom_cfg = MomTop1Config(
    lookback=20,
    mom_low=0.05,
    mom_high=0.3,
    min_hold_days=3,
    take_profit=0.5,
    tp_cooldown_days=6,
)

pack = build_mom_threshold_top1_weights_next_open(md, mom_cfg)
w_strat = pack["weights"]

out = backtest_weights(
    md=md,
    weights=w_strat,
    entry_mode=cfg.entry_mode,
    fee_bps=cfg.fee_bps,
    rf_annual=cfg.rf_annual,
    long_only=True,
    allow_leverage=False,
    max_gross=1.01,
)

out_bench = backtest_weights(
    md=md,
    weights=build_benchmark_weights(md.close, cfg),
    fee_bps=0,
    rf_annual=0,
    long_only=True,
    allow_leverage=False,
    max_gross=1.01,
)

# diagnostics you will want:
top1_table = pack["top1_table"]
mom = pack["mom"]

In [ ]:
plot_compare(
    out,
    out_bench,
    ''
)